In [4]:
import pandas as pd
import numpy as np
import networkx as nx
import osmnx as ox
# import cplex

In [ ]:
import datetime

In [ ]:
!pip install cplex

In [1]:
pip install numpy==1.23

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
pip install pandas==1.5.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
pip install osmnx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from zipfile import ZipFile
file_name = "OneDrive_2023-03-29.zip"

with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('Done')

Done


In [5]:
trip_data = pd.read_csv(r"/content/trips.txt")
stops_data = pd.read_csv(r"/content/stops.txt")
stops_times_data = pd.read_csv(r"/content/stop_times.txt")
    # routes_data = pd.read_csv(r"/content/128 Business Council GTFS/routes.txt")

In [ ]:
stops_times_data

In [ ]:
result2 = stops_times_data.groupby('trip_id').agg(
    {
        'stop_id': ['first', 'last'], 
        'arrival_time': 'last', 
        'departure_time': 'first'
    }
)

In [ ]:
result2[("arrival_time",'last')] = pd.to_datetime(result2[("arrival_time",'last')],errors = "coerce")
result2[("departure_time",'first')] = pd.to_datetime(result2[("departure_time",'first')],errors = "coerce")

In [ ]:
result2 = result2.reset_index()

In [ ]:
depot = result2[('stop_id', 'first')].value_counts().reset_index()

In [ ]:
print("depot is stop number ", depot["index"][0] )

depot is stop number  807645


In [ ]:
origin_with_depot = result2.loc[result2[('stop_id', 'first')] == depot["index"][0]].index.tolist()
origin_with_depot
# for these trips deadhead will be zero so c_74_2 will be zero


In [ ]:
origin_without_depot = result2.loc[result2[('stop_id', 'first')] != depot["index"][0]].index.tolist()

In [ ]:
dest_with_depot = result2.loc[result2[('stop_id', 'last')] == depot["index"][0]].index.tolist()
dest_with_depot
# for these trips deadhead will be zero so c_0_74 will be zero

In [ ]:
unique_values = pd.concat([result2[('stop_id', 'first')], result2[('stop_id', 'last')]]).unique().tolist()
# unique_values

In [ ]:
li1 = []
df = stops_data[stops_data["stop_id"].isin(unique_values)]
for idd, row in df.iterrows():
    li1.append((row.stop_id,row.stop_lon,row.stop_lat))

In [ ]:
# li1

In [11]:
G = ox.graph_from_place("Bangalore India", network_type="drive")

In [ ]:
location_point = (12.969722,77.505573) # Bangalore city center
distance = 5000 # meters
G = ox.graph_from_point(location_point, distance=distance, network_type='drive')

# Define the target point for which we want to find the 2nd nearest node
target_point = (12.9779, 77.6080)

# Find the nearest node to the target point
nearest_node = ox.get_nearest_node(G, target_point)

# Find all the neighboring nodes of the nearest node
neighbors = list(G.neighbors(nearest_node))

# Calculate the distance between the target point and each neighboring node
distances = []
for neighbor in neighbors:
    neighbor_point = ox.get_node(G, neighbor)
    distance = ox.distance.euclidean_dist_vec(target_point[0], target_point[1], neighbor_point[0], neighbor_point[1])
    distances.append(distance)

# Sort the neighboring nodes based on their distance from the target point
sorted_neighbors = [x for _,x in sorted(zip(distances,neighbors))]

# Select the 2nd nearest node
second_nearest_node = sorted_neighbors[1]

In [ ]:
# G = ox.project_graph(G)
stop_node = {}
for i in range(len(li1)):
  stop_node[li1[i][0]] = ox.nearest_nodes(G, li1[i][1], li1[i][2])

In [ ]:
df = stops_data.drop_duplicates(subset=['stop_id'])
li1 = []
for idd, row in df.iterrows():
    li1.append((row.stop_id, row.stop_lon, row.stop_lat))
    
stop_node = {}
for i in range(len(li1)):
  stop_node[li1[i][0]] = ox.nearest_nodes(G, li1[i][1], li1[i][2])

In [ ]:
li1

In [ ]:
distance = {}

for i in stop_node:
  distance[i] = {}
  for j in stop_node:
              distance[i][j] = (nx.shortest_path_length(G, source = stop_node[i], target = stop_node[j], weight='length', method ='dijkstra'))


# velocity = 30

In [15]:
distance

{3: {3: 0,
  5: 13729.509000000005,
  6: 5821.715999999998,
  7: 7570.932000000003,
  8: 12550.089,
  9: 12852.515,
  10: 11990.994,
  12: 11418.791000000001,
  17: 10365.386000000002,
  18: 10011.094000000001,
  20: 9293.057000000003,
  21: 8868.297,
  22: 8910.552000000001,
  23: 8480.153000000004,
  25: 8676.283000000005,
  27: 8148.488000000005,
  28: 8148.488000000005,
  29: 7552.628000000002,
  30: 13791.965999999997,
  31: 10968.732,
  32: 8829.065999999999,
  35: 8781.076000000003,
  36: 9015.329000000003,
  37: 10232.127000000004,
  38: 11077.584000000004,
  39: 10574.873000000001,
  40: 12012.237000000003,
  41: 13204.219000000001,
  42: 15354.040000000003,
  43: 15354.040000000003,
  44: 15354.040000000003,
  45: 12192.438000000002,
  46: 10232.127000000004,
  47: 8986.370999999997,
  48: 9283.887000000004,
  49: 9453.558,
  50: 7078.076000000002}}

In [ ]:
trips_origin = []
for i in origin_without_depot:
  dest = result2.iloc[i]['stop_id']['first']
  origin = depot["index"][0]
  trips_origin.append((len(result2),i,distance[origin][dest]))


In [ ]:
trips_origin

[(74, 0, 765.0523409363744),
 (74, 1, 589.4334933973589),
 (74, 2, 771.6585834333735),
 (74, 6, 853.585354141657),
 (74, 8, 1070.4189675870348),
 (74, 10, 798.6585834333736),
 (74, 13, 853.585354141657),
 (74, 14, 916.0198079231692),
 (74, 15, 771.6585834333735),
 (74, 17, 798.6585834333736),
 (74, 18, 589.4334933973589),
 (74, 19, 479.0136854741898),
 (74, 20, 853.585354141657),
 (74, 21, 884.1418967587038),
 (74, 22, 765.0523409363744),
 (74, 23, 624.9295318127251),
 (74, 24, 798.6585834333736),
 (74, 25, 909.3637454981994),
 (74, 27, 727.6501800720289),
 (74, 29, 771.6585834333735),
 (74, 31, 727.6501800720289),
 (74, 32, 909.3637454981994),
 (74, 33, 1040.9997599039616),
 (74, 34, 589.4334933973589),
 (74, 35, 731.5558223289316),
 (74, 36, 1070.4189675870348),
 (74, 38, 1070.4189675870348),
 (74, 41, 589.4334933973589),
 (74, 42, 916.0198079231692),
 (74, 43, 727.6501800720289),
 (74, 45, 479.0136854741898),
 (74, 46, 479.0136854741898),
 (74, 47, 589.4334933973589),
 (74, 52, 765.

In [ ]:
# distance

In [ ]:
trips = []
for index1, row1 in result2.iterrows():
#     t1_first_stop = row1['stop_id']['first']
    t1_last_stop = row1['stop_id']['last']
    t1_arrival_time = row1['arrival_time']['last']
    for index2, row2 in result2.iterrows():
        t2_first_stop = row2['stop_id']['first']
#         t2_last_stop = row2['stop_id']['last']
        t2_departure_time = row2['departure_time']['first']
        if index1 != index2:
            # print(t1_last_stop)
            # print(t2_first_stop)
            if t1_arrival_time + datetime.timedelta(seconds=distance[t1_last_stop][t2_first_stop]) <= t2_departure_time:
                trips.append((index1,index2,distance[t1_last_stop][t2_first_stop]))
            
            

# optimization model

# model for bus minimization

In [ ]:
model = cplex.Cplex()
model.objective.set_sense(model.objective.sense.minimize)

In [ ]:
sorted_trips = sorted(trips, key=lambda x: x[1])

In [ ]:
# sorted_trips

In [ ]:
dec_var = ["x"+str(row[0])+"_"+str(row[1]) for row in trips]
# dec_var = []
for row in origin_with_depot:
  dec_var.append("x"+str(len(result2))+"_"+str(row))
for row in dest_with_depot:
  dec_var.append("x"+str(row)+"_"+str(len(result2)))
for row in trips_origin:
  dec_var.append("x"+str(row[0])+"_"+str(row[1]))
# decision variable

In [ ]:
obj = [1 for row in dec_var]

In [ ]:
lb = [0 for i in range(len(dec_var))] # lower bound
ub = [1 for i in range(len(dec_var))] # upper bound
type_int =  ["I" for i in range(len(dec_var))]

In [ ]:
model.variables.add(obj=obj, lb=lb, ub=ub, names=dec_var, types= type_int)

range(0, 2201)

In [ ]:
constraint_names = ["c1"]
first_constraint = [["x"+str(len(result2))+"_"+str(row) for row in origin_with_depot] 
                    + ["x"+str(row[0])+"_"+str(row[1]) for row in trips_origin],
                    [1 for row in origin_with_depot]+[1 for row in trips_origin]]
constraints = [first_constraint]
constraint_direction = ["L"]
rhs = [24]

model.linear_constraints.add(lin_expr=constraints, senses=constraint_direction, rhs=rhs, names=constraint_names)

range(0, 1)

In [ ]:
prev_node = trips[0][0]
k=3
constraints_out = {}
for i in range(len(result2)):
  constraints_out["constraint_"+str(i)] = [[],[]]
for row in trips:
  
  if prev_node == row[0]:
    constraints_out["constraint_"+str(row[0])][0].append("x"+str(row[0])+"_"+str(row[1]))
    constraints_out["constraint_"+str(row[0])][1].append(1)
  else:
    constraints_out["constraint_"+str(row[0])][0].append("x"+str(row[0])+"_"+str(row[1]))
    constraints_out["constraint_"+str(row[0])][1].append(1)
    if prev_node in dest_with_depot:
      constraints_out["constraint_"+str(prev_node)][0].append("x"+str(prev_node)+"_"+str(len(result2)))
      constraints_out["constraint_"+str(prev_node)][1].append(1)
    prev_node = row[0]

if trips[-1][0] in dest_with_depot:
    constraints_out["constraint_"+str(trips[-1][0])][0].append("x"+str(trips[-1][0])+"_"+str(len(result2)))
    constraints_out["constraint_"+str(trips[-1][0])][1].append(1)

for trip in sorted_trips:
  if trip[1] in dest_with_depot and len(constraints_out["constraint_"+str(trip[1])][0])== 0:
    # print(trip)
    constraints_out["constraint_"+str(trip[1])][0].append("x"+str(trip[1])+"_"+str(len(result2)))
    constraints_out["constraint_"+str(trip[1])][1].append(1)

c_out = [constraints_out["constraint_"+str(row)] for row in range(len(constraints_out)) if constraints_out["constraint_"+str(row)][0] != []]
constraint_direction = []
rhs = []
constraint_names = []
for i in range(3,len(c_out)+3):
  constraint_names.append("c_out"+str(i))
for i in range(len(c_out)):
  constraint_direction.append("E")
  rhs.append(1)
model.linear_constraints.add(lin_expr=c_out, senses=constraint_direction, rhs=rhs, names=constraint_names)


range(1, 75)

In [ ]:
x_ij_names = []
x_hi_names = []
for row in origin_with_depot:
  x_ij_names.append("x"+str(len(result2))+"_"+str(row))
for row in trips_origin:
  x_ij_names.append("x"+str(row[0])+"_"+str(row[1]))
for row in dest_with_depot:
  x_hi_names.append("x"+str(row)+"_"+str(len(result2)))


In [ ]:
demand_constaint = cplex.SparsePair(ind=x_ij_names+x_hi_names, val=[1.0]*len(x_ij_names) + [-1.0]*len(x_hi_names))
model.linear_constraints.add(lin_expr=[demand_constaint], senses=["E"], rhs=[0.0],names = ["c2"])

range(75, 76)

In [ ]:
# x_ij_names

In [ ]:
# sorted_trips = sorted(trips, key=lambda x: x[1])

In [ ]:
prev_node = trips[1][0]
k=3
constraints_in = {}
for i in range(len(result2)):
  constraints_in["constraint_"+str(i)] = [[],[]]
for row in sorted_trips:
  if prev_node == row[1]:
    constraints_in["constraint_"+str(row[1])][0].append("x"+str(row[0])+"_"+str(row[1]))
    constraints_in["constraint_"+str(row[1])][1].append(1)
  else:
    constraints_in["constraint_"+str(row[1])][0].append("x"+str(row[0])+"_"+str(row[1]))
    constraints_in["constraint_"+str(row[1])][1].append(1)
    if prev_node in origin_with_depot:
      constraints_in["constraint_"+str(prev_node)][0].append("x"+str(len(result2))+"_"+str(prev_node))
      constraints_in["constraint_"+str(prev_node)][1].append(1)
    prev_node = row[1]
if sorted_trips[-1][1] in origin_with_depot:
    constraints_in["constraint_"+str(sorted_trips[-1][1])][0].append("x"+str(len(result2))+"_"+str(sorted_trips[-1][1]))
    constraints_in["constraint_"+str(sorted_trips[-1][1])][1].append(1)

for trip in trips:
  if trip[0] in origin_with_depot and len(constraints_in["constraint_"+str(trip[0])][0])== 0:
    print(trip)
    constraints_in["constraint_"+str(trip[0])][0].append("x"+str(len(result2))+"_"+str(str(trip[0])))
    constraints_in["constraint_"+str(trip[0])][1].append(1)
c_in = [constraints_in["constraint_"+str(row)] for row in range(len(constraints_in)) if constraints_in["constraint_"+str(row)][0] != []]

for trip in trips_origin:
  # if trip[0] in origin_with_depot and len(constraints_in["constraint_"+str(trip[0])][0])== 0:
  #   print(trip)
    constraints_in["constraint_"+str(trip[1])][0].append("x"+str(len(result2))+"_"+str(str(trip[1])))
    constraints_in["constraint_"+str(trip[1])][1].append(1)
c_in = [constraints_in["constraint_"+str(row)] for row in range(len(constraints_in)) if constraints_in["constraint_"+str(row)][0] != []]
constraint_direction = []
rhs = []
constraint_names = []
for i in range(3,len(c_in)+3):
  constraint_names.append("c_in"+str(i))
for i in range(len(c_in)):
  constraint_direction.append("E")
  rhs.append(1)



(5, 0, 765.0523409363744)
(11, 0, 765.0523409363744)
(51, 0, 765.0523409363744)
(60, 0, 371.7049219687875)
(68, 0, 765.0523409363744)
(72, 0, 33.49651860744298)


In [ ]:
# c_in

In [ ]:
model.linear_constraints.add(lin_expr=c_in, senses=constraint_direction, rhs=rhs, names=constraint_names)

range(76, 150)

In [ ]:
flow_conservation = {}
for i in range(len(result2)):
  flow_conservation[i] = []
# for i in range(len(result2)):
#   constraints_out["constraint_"+str(i)] = [[],[]]
for i in range(len(result2)):
  if "constraint_"+str(i) in constraints_out and "constraint_"+str(i) in constraints_in:
    flow_conservation[i].append(constraints_out["constraint_"+str(i)][0]+ constraints_in["constraint_"+str(i)][0])
    flow_conservation[i].append(constraints_out["constraint_"+str(i)][1] + [-x for x in constraints_in["constraint_"+str(i)][1]])

c_flow = [flow_conservation[row] for row in range(len(flow_conservation))if flow_conservation[row][0] != []]
constraint_direction = []
rhs = []
constraint_names = []
for i in range(3,len(c_flow)+3):
  constraint_names.append("c_flow"+str(i))
for i in range(len(c_flow)):
  constraint_direction.append("E")
  rhs.append(0)
model.linear_constraints.add(lin_expr=c_flow, senses=constraint_direction, rhs=rhs, names=constraint_names)

range(150, 224)

In [ ]:
model.write("example1.lp")

In [ ]:
model = cplex.Cplex("example1.lp")
# model.solve()

In [ ]:
# print(model.solution.get_values())

In [ ]:
# print(model.solution.get_objective_value())

In [ ]:
model.solution.write("output1.sol")

# model for cost *minimization*

In [ ]:
model = cplex.Cplex()
model.objective.set_sense(model.objective.sense.minimize)

In [ ]:
sorted_trips = sorted(trips, key=lambda x: x[1])

In [ ]:
# sorted_trips

In [ ]:
dec_var = ["x"+str(row[0])+"_"+str(row[1]) for row in trips]
# dec_var = []
for row in origin_with_depot:
  dec_var.append("x"+str(len(result2))+"_"+str(row))
for row in dest_with_depot:
  dec_var.append("x"+str(row)+"_"+str(len(result2)))
for row in trips_origin:
  dec_var.append("x"+str(row[0])+"_"+str(row[1]))
# decision variable

In [ ]:
obj = [row[2] for row in trips]
for row in origin_with_depot:
  obj.append(0)
for row in dest_with_depot:
  obj.append(0)
for row in trips_origin:
  obj.append(row[2])
# objective

In [ ]:
lb = [0 for i in range(len(dec_var))] # lower bound
ub = [1 for i in range(len(dec_var))] # upper bound
type_int =  ["I" for i in range(len(dec_var))]

In [ ]:
model.variables.add(obj=obj, lb=lb, ub=ub, names=dec_var, types= type_int)

range(0, 2201)

#capacity

In [ ]:
constraint_names = ["c1"]
first_constraint = [["x"+str(len(result2))+"_"+str(row) for row in origin_with_depot] 
                    + ["x"+str(row[0])+"_"+str(row[1]) for row in trips_origin],
                    [1 for row in origin_with_depot]+[1 for row in trips_origin]]
constraints = [first_constraint]
constraint_direction = ["L"]
rhs = [24]

model.linear_constraints.add(lin_expr=constraints, senses=constraint_direction, rhs=rhs, names=constraint_names)

range(0, 1)

#outgoing flow

In [ ]:
prev_node = trips[0][0]
k=3
constraints_out = {}
for i in range(len(result2)):
  constraints_out["constraint_"+str(i)] = [[],[]]
for row in trips:
  
  if prev_node == row[0]:
    constraints_out["constraint_"+str(row[0])][0].append("x"+str(row[0])+"_"+str(row[1]))
    constraints_out["constraint_"+str(row[0])][1].append(1)
  else:
    constraints_out["constraint_"+str(row[0])][0].append("x"+str(row[0])+"_"+str(row[1]))
    constraints_out["constraint_"+str(row[0])][1].append(1)
    if prev_node in dest_with_depot:
      constraints_out["constraint_"+str(prev_node)][0].append("x"+str(prev_node)+"_"+str(len(result2)))
      constraints_out["constraint_"+str(prev_node)][1].append(1)
    prev_node = row[0]

if trips[-1][0] in dest_with_depot:
    constraints_out["constraint_"+str(trips[-1][0])][0].append("x"+str(trips[-1][0])+"_"+str(len(result2)))
    constraints_out["constraint_"+str(trips[-1][0])][1].append(1)

for trip in sorted_trips:
  if trip[1] in dest_with_depot and len(constraints_out["constraint_"+str(trip[1])][0])== 0:
    # print(trip)
    constraints_out["constraint_"+str(trip[1])][0].append("x"+str(trip[1])+"_"+str(len(result2)))
    constraints_out["constraint_"+str(trip[1])][1].append(1)

c_out = [constraints_out["constraint_"+str(row)] for row in range(len(constraints_out)) if constraints_out["constraint_"+str(row)][0] != []]
constraint_direction = []
rhs = []
constraint_names = []
for i in range(3,len(c_out)+3):
  constraint_names.append("c_out"+str(i))
for i in range(len(c_out)):
  constraint_direction.append("E")
  rhs.append(1)
model.linear_constraints.add(lin_expr=c_out, senses=constraint_direction, rhs=rhs, names=constraint_names)


range(1, 75)

# depot flow conservation

In [ ]:
x_ij_names = []
x_hi_names = []
for row in origin_with_depot:
  x_ij_names.append("x"+str(len(result2))+"_"+str(row))
for row in trips_origin:
  x_ij_names.append("x"+str(row[0])+"_"+str(row[1]))
for row in dest_with_depot:
  x_hi_names.append("x"+str(row)+"_"+str(len(result2)))


In [ ]:
demand_constaint = cplex.SparsePair(ind=x_ij_names+x_hi_names, val=[1.0]*len(x_ij_names) + [-1.0]*len(x_hi_names))
model.linear_constraints.add(lin_expr=[demand_constaint], senses=["E"], rhs=[0.0],names = ["c2"])

range(75, 76)

# incoming flows

In [ ]:
prev_node = trips[1][0]
k=3
constraints_in = {}
for i in range(len(result2)):
  constraints_in["constraint_"+str(i)] = [[],[]]
for row in sorted_trips:
  if prev_node == row[1]:
    constraints_in["constraint_"+str(row[1])][0].append("x"+str(row[0])+"_"+str(row[1]))
    constraints_in["constraint_"+str(row[1])][1].append(1)
  else:
    constraints_in["constraint_"+str(row[1])][0].append("x"+str(row[0])+"_"+str(row[1]))
    constraints_in["constraint_"+str(row[1])][1].append(1)
    if prev_node in origin_with_depot:
      constraints_in["constraint_"+str(prev_node)][0].append("x"+str(len(result2))+"_"+str(prev_node))
      constraints_in["constraint_"+str(prev_node)][1].append(1)
    prev_node = row[1]
if sorted_trips[-1][1] in origin_with_depot:
    constraints_in["constraint_"+str(sorted_trips[-1][1])][0].append("x"+str(len(result2))+"_"+str(sorted_trips[-1][1]))
    constraints_in["constraint_"+str(sorted_trips[-1][1])][1].append(1)

for trip in trips:
  if trip[0] in origin_with_depot and len(constraints_in["constraint_"+str(trip[0])][0])== 0:
    print(trip)
    constraints_in["constraint_"+str(trip[0])][0].append("x"+str(len(result2))+"_"+str(str(trip[0])))
    constraints_in["constraint_"+str(trip[0])][1].append(1)
c_in = [constraints_in["constraint_"+str(row)] for row in range(len(constraints_in)) if constraints_in["constraint_"+str(row)][0] != []]

for trip in trips_origin:
  # if trip[0] in origin_with_depot and len(constraints_in["constraint_"+str(trip[0])][0])== 0:
  #   print(trip)
    constraints_in["constraint_"+str(trip[1])][0].append("x"+str(len(result2))+"_"+str(str(trip[1])))
    constraints_in["constraint_"+str(trip[1])][1].append(1)
c_in = [constraints_in["constraint_"+str(row)] for row in range(len(constraints_in)) if constraints_in["constraint_"+str(row)][0] != []]
constraint_direction = []
rhs = []
constraint_names = []
for i in range(3,len(c_in)+3):
  constraint_names.append("c_in"+str(i))
for i in range(len(c_in)):
  constraint_direction.append("E")
  rhs.append(1)



(5, 0, 765.0523409363744)
(11, 0, 765.0523409363744)
(51, 0, 765.0523409363744)
(60, 0, 371.7049219687875)
(68, 0, 765.0523409363744)
(72, 0, 33.49651860744298)


In [ ]:
model.linear_constraints.add(lin_expr=c_in, senses=constraint_direction, rhs=rhs, names=constraint_names)

range(76, 150)

# flow conservation


In [ ]:
flow_conservation = {}
for i in range(len(result2)):
  flow_conservation[i] = []
# for i in range(len(result2)):
#   constraints_out["constraint_"+str(i)] = [[],[]]
for i in range(len(result2)):
  if "constraint_"+str(i) in constraints_out and "constraint_"+str(i) in constraints_in:
    flow_conservation[i].append(constraints_out["constraint_"+str(i)][0]+ constraints_in["constraint_"+str(i)][0])
    flow_conservation[i].append(constraints_out["constraint_"+str(i)][1] + [-x for x in constraints_in["constraint_"+str(i)][1]])

c_flow = [flow_conservation[row] for row in range(len(flow_conservation))if flow_conservation[row][0] != []]
constraint_direction = []
rhs = []
constraint_names = []
for i in range(3,len(c_flow)+3):
  constraint_names.append("c_flow"+str(i))
for i in range(len(c_flow)):
  constraint_direction.append("E")
  rhs.append(0)
model.linear_constraints.add(lin_expr=c_flow, senses=constraint_direction, rhs=rhs, names=constraint_names)

range(150, 224)

In [ ]:
model.write("example2.lp")

In [ ]:
model = cplex.Cplex("example2.lp")
model.solve()

In [ ]:
model.solution.write("output2.sol")

In [ ]:
G = nx.DiGraph()
# G.add_node("a", demand=-5)
# G.add_node("d", demand=5)
# G.add_edge("a", "b", weight=3, capacity=4)
# G.add_edge("a", "c", weight=6, capacity=10)
# G.add_edge("b", "d", weight=1, capacity=9)
# G.add_edge("c", "d", weight=2, capacity=5)
# flowDict = nx.min_cost_flow(G)

In [ ]:
G = nx.DiGraph()

G.add_node(len(result2), demand=-12)
G.add_node(len(result2)+1,demand = 12)
for row in trips:
  G.add_edge(row[0], row[1], weight=1, capacity=1)
for row in origin_with_depot:
  G.add_edge(len(result2), row, weight=0, capacity=1)
for row in dest_with_depot:
  G.add_edge(row,len(result2)+1, weight=0, capacity=1)
for row in trips_origin:
  G.add_edge(len(result2),row[1], weight=1, capacity=1)


In [ ]:
flowDict = nx.min_cost_flow(G)

In [ ]:
flowDict

In [ ]:
G = nx.DiGraph()

G.add_node(len(result2), demand=-17)
G.add_node(len(result2)+1,demand = 17)
for row in trips:
  G.add_edge(row[0], row[1], weight=row[2], capacity=1)
for row in origin_with_depot:
  G.add_edge(len(result2), row, weight=0, capacity=1)
for row in dest_with_depot:
  G.add_edge(row,len(result2)+1, weight=0, capacity=1)
for row in trips_origin:
  G.add_edge(len(result2),row[1], weight=row[2], capacity=1)

In [ ]:
flowDict = nx.min_cost_flow(G)

In [ ]:
flowDict